### ![](../../img/installation-ico.png) Installation 

In [ ]:
pip install langchain langchain-community langchain-google-vertexai google.auth

Fonctionnalités nécessaires à la récupération de données web (scraping)

In [ ]:
pip install beautifulsoup4

### ![](../../img/package-ico.png) Imports

In [1]:
import os
import google.auth

from langchain_google_vertexai import VertexAI

# pour la récupération de données depuis une page web
from langchain_community.document_loaders import WebBaseLoader

# pour le templeting des messages du prompt d'entrée
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage
from langchain.prompts import HumanMessagePromptTemplate

# pour le parser de sortie
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

e:\Projets\Sfeir\sfeir-school-langchain\.venv.school\Lib\site-packages\google_crc32c\__init__.py:29: RuntimeWarning: As the c extension couldn't be imported, `google-crc32c` is using a pure python implementation that is significantly slower. If possible, please configure a c build environment and compile the extension
  warnings.warn(_SLOW_CRC32C_WARNING, RuntimeWarning)
USER_AGENT environment variable not set, consider setting it to identify your requests.


### ![](../../img/parametrage-ico.png) Paramétrages 

In [2]:
# Gestion credentials VertexAI

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../settings/credentials.json"
google.auth.default()

(<google.oauth2.service_account.Credentials at 0x19cc53a5b20>,
 'fit-symbol-433212-u5')

In [3]:
# Étape 1 : Charger le document avec un DataLoader
url = "https://newsroom.sfeir.com/mascotte-sfeir-pourquoi-avoir-choisi-un-axolotl-"
loader = WebBaseLoader(url)
documents = loader.load()
document_content = documents[0].page_content

print(document_content)







Mascotte SFEIR : Pourquoi avoir choisi un axolotl ?



























































Accueil
SFEIR School
SFEIR Institute








































Actualités
Événements
SFEIR Recrute !











Actualités
Événements
SFEIR Recrute !



































Mascotte SFEIR : Pourquoi avoir choisi un axolotl ?

Publié par
SFEIR Team le 21 mars 2019
  

Suivez moi sur :

LinkedIn
Twitter 









Tweet














L’axolotl est l’animal totem de SFEIR depuis 2016. Fétiche de l’entreprise, sa bouille bienveillante séduit en interne comme en externe. Mais alors, pourquoi avoir choisi cette créature ? Petit retour sur la genèse de notre mascotte.
Une initiative interne
L’idée d’avoir une mascotte pour l’entreprise avait germée quelques temps auparavant. C’est en effet monnaie courante dans notre milieu : DinoChrome, Octocat - GitHub, Docker, Golang, pour n’en citer que quelques unes, font l’unanimité : différents produits à leurs e

In [4]:
# Étape 2 : Définir le modèle de réponse avec Pydantic
class Response(BaseModel):
    answer: str = Field(description="Answer to the question")


# Étape 3 : Définir le JsonOutputParser
parser = JsonOutputParser(pydantic_object=Response)


# Étape 4 : Définir le LLM
llm = VertexAI()

In [5]:
# Étape 5 : Définir le ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        # Message système pour spécifier le comportement
        SystemMessage(content=("You are a helpful assistant.")),
        # Message système précisant le format de sortie
        SystemMessage(content=(parser.get_format_instructions())),
        # Message humain à partir d'un template
        HumanMessagePromptTemplate.from_template(
            """Answer the question based only on the following context:
            {context}

            Question: {question}
            """
        ),
    ]
)

In [7]:
# Étape 6 : Chaîner les éléments
chain = prompt | llm | parser

In [8]:
# Invoquer la chaîne en spécifiant le contexte et la question
result = chain.invoke(
    {
        "context": documents,
        "question": "Résumes moi ce document en une seule phrase",
    }
)
print(result)

{'answer': "L'axolotl a été choisi comme mascotte de SFEIR en 2016 en raison de ses caractéristiques qui reflètent les valeurs de l'entreprise, telles que la jeunesse éternelle, la capacité d'adaptation et la régénération."}
